In [17]:
import json

def get_date(normalized_data, target_page, tolerance=0.01):
    with open(normalized_data, 'r') as file:
        data = json.load(file)

    x_start = float('inf')
    date = []
    
    for page in data:
        if page['page_id'] == target_page and 'words' in page:
            for word in page['words']:
                x_coordinate = word['geometry'][0][0]
                y_coordinate = word['geometry'][0][1]
                if x_coordinate < x_start:
                    x_start = x_coordinate

                if abs(x_coordinate - x_start) <= tolerance:
                    date.append((word['value'], x_coordinate, y_coordinate))

    return  date


In [18]:
normalized_data = 'sample/normalized/mandiri-transaction_inquery.json'
target_page = 0
date = get_date(normalized_data, target_page)
date

[('02/06/2022', 0.05930861928104575, 0.3662109375),
 ('03/06/2022', 0.05930861928104575, 0.3857421875),
 ('03/06/2022', 0.05930861928104575, 0.4052734375),
 ('06/06/2022', 0.05930861928104575, 0.4345703125),
 ('06/06/2022', 0.05930861928104575, 0.46484375),
 ('07/06/2022', 0.05930861928104575, 0.494140625),
 ('07/06/2022', 0.05930861928104575, 0.5244140625),
 ('08/06/2022', 0.05930861928104575, 0.54296875),
 ('08/06/2022', 0.05930861928104575, 0.5732421875),
 ('09/06/2022', 0.05930861928104575, 0.591796875),
 ('09/06/2022', 0.05796504799836599, 0.6220703125),
 ('09/06/2022', 0.05796504799836599, 0.662109375),
 ('09/06/2022', 0.05796504799836599, 0.6923828125),
 ('10/06/2022', 0.05930861928104575, 0.7216796875),
 ('10/06/2022', 0.05930861928104575, 0.7509765625),
 ('10/06/2022', 0.05930861928104575, 0.7705078125),
 ('10/06/2022', 0.05930861928104575, 0.7900390625),
 ('13/06/2022', 0.05930861928104575, 0.830078125),
 ('13/06/2022', 0.05930861928104575, 0.849609375)]